In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torch.optim import Adam
from model import CharRNN
from torch.nn import functional as F
from torch import distributions

%load_ext autoreload
%autoreload 2

## Step 0: Prepare dataset

In [2]:
with open('/media/junhong/LargeStorage/arXiv-papers/papers/raw_latex.txt', 'r') as f:
    raw_data = f.read()

In [3]:
raw_data = raw_data.encode('utf-8').decode('utf-8')[:15000000]
characters = set(raw_data)
num_char = len(characters)
print('Length of the file', len(raw_data))
print('Character set:', characters)
print('Number of characters:', num_char)

Length of the file 15000000
Character set: {'t', 'N', 'E', '8', 'u', '7', '[', 'H', '!', 'A', ']', '?', 'I', '"', ')', '9', '_', 'j', '}', '.', 'C', '/', 'x', 'd', 'D', '-', 'R', 'X', '{', 'c', '@', 'Q', 'f', '(', ' ', '|', 'r', '\t', 'l', 'n', 'v', ';', '&', 'k', 'V', 'P', 'p', 'm', 'L', '^', '`', 'a', '$', 'y', '\\', '4', '%', 'o', 'q', 'e', 'W', '2', 'J', '3', 's', '0', 'F', 'S', '<', 'Z', 'B', 'U', '=', ',', 'T', '*', 'b', 'M', '~', "'", 'w', 'K', '\n', ':', '5', '#', 'Y', '1', '>', 'O', 'z', '+', 'G', 'i', 'h', 'g', '6'}
Number of characters: 97


In [4]:
# define our character to index dictionary
char_idx = {k: v for (v, k) in enumerate(characters)}
idx_char = {char_idx[k]: k for k in char_idx.keys()}
print('Character to index:', char_idx)
print('Index to character:', idx_char)

Character to index: {'t': 0, 'N': 1, 'E': 2, '8': 3, 'u': 4, '7': 5, '[': 6, 'H': 7, '!': 8, 'A': 9, ']': 10, '?': 11, 'I': 12, '"': 13, ')': 14, '9': 15, '_': 16, 'j': 17, '}': 18, '.': 19, 'C': 20, '/': 21, 'x': 22, 'd': 23, 'D': 24, '-': 25, 'R': 26, 'X': 27, '{': 28, 'c': 29, '@': 30, 'Q': 31, 'f': 32, '(': 33, ' ': 34, '|': 35, 'r': 36, '\t': 37, 'l': 38, 'n': 39, 'v': 40, ';': 41, '&': 42, 'k': 43, 'V': 44, 'P': 45, 'p': 46, 'm': 47, 'L': 48, '^': 49, '`': 50, 'a': 51, '$': 52, 'y': 53, '\\': 54, '4': 55, '%': 56, 'o': 57, 'q': 58, 'e': 59, 'W': 60, '2': 61, 'J': 62, '3': 63, 's': 64, '0': 65, 'F': 66, 'S': 67, '<': 68, 'Z': 69, 'B': 70, 'U': 71, '=': 72, ',': 73, 'T': 74, '*': 75, 'b': 76, 'M': 77, '~': 78, "'": 79, 'w': 80, 'K': 81, '\n': 82, ':': 83, '5': 84, '#': 85, 'Y': 86, '1': 87, '>': 88, 'O': 89, 'z': 90, '+': 91, 'G': 92, 'i': 93, 'h': 94, 'g': 95, '6': 96}
Index to character: {0: 't', 1: 'N', 2: 'E', 3: '8', 4: 'u', 5: '7', 6: '[', 7: 'H', 8: '!', 9: 'A', 10: ']', 11:

In [5]:
# convert raw data to character index
processed_data = [int(char_idx[c]) for c in raw_data]
print('Partial raw data', raw_data[:10])
print('Processed data', processed_data[:10])

Partial raw data \documentc
Processed data [54, 23, 57, 29, 4, 47, 59, 39, 0, 29]


In [6]:
# here we define our method for getting the data
def get_data(batch_size = 256, chunk_len=15):
    # input tensor
    x = torch.LongTensor(batch_size, chunk_len)
    # target tensor
    y = torch.LongTensor(batch_size, chunk_len)
    
    for batch in range(batch_size):
        # randomly choose an index to start
        start_idx = np.random.randint(low=0, high=len(raw_data) - chunk_len)
        data = processed_data[start_idx: start_idx + chunk_len + 1]
        x[batch] = torch.LongTensor(data[:-1])
        y[batch] = torch.LongTensor(data[1:])
    return Variable(x).cuda(), Variable(y).cuda()

In [7]:
# test our get_data function
x, y = get_data(2, 5)
print(x)
print(y)
for inpt, target in zip(x, y):
    inpt = inpt.data.cpu().numpy()
    target = target.data.cpu().numpy()
    print('Input of this batch is "%s"' %(''.join([idx_char[i] for i in inpt])))
    print('Target of this batch is "%s"' %(''.join([idx_char[i] for i in target])))

Variable containing:
 16  87  49  54  64
 76  38  59  34  29
[torch.cuda.LongTensor of size 2x5 (GPU 0)]

Variable containing:
 87  49  54  64  46
 38  59  34  29  57
[torch.cuda.LongTensor of size 2x5 (GPU 0)]

Input of this batch is "_1^\s"
Target of this batch is "1^\sp"
Input of this batch is "ble c"
Target of this batch is "le co"


## Step 1: Define training function

In [8]:
def train(rnn, lr=1e-3, epoch=300, batch_size=512, chunck_len=200):
    iter_per_epoch = len(raw_data)//batch_size
    optimizer = Adam(rnn.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for e in range(epoch):
        for i in range(iter_per_epoch):
            rnn.train()
            inpt, target = get_data(batch_size, chunk_len=chunck_len)
            
            # forward
            hidden = rnn.init_hidden(batch_size)
            output, hidden = rnn.forward(inpt, hidden)
            
            loss = criterion(output, target.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 500 == 0:
                print('At training iteration: %i, loss is %.4f' %(i, loss.data[0]))
            # sample
            if i % 1000 == 0:
                # evaluation
                rnn.eval()
                target = target.cpu().data.numpy()[0]
                output = output.view(batch_size, chunck_len, -1)
                output = F.softmax(output, dim=2)
                output = output.data.cpu().numpy()
                output = output.reshape(batch_size, chunck_len, num_char)
                output = output[0]
                content = ''
                for o in output:
                    content += idx_char[np.random.choice(np.arange(num_char), p=o)]
                print('-------------Start Evaluation-----------------------')
                print('Sampled content:', content)
                print('Target:', ''.join([idx_char[i] for i in target]))
                print('-------------End Evaluation-------------------------')
        torch.save(rnn.state_dict(), 'net.pth')

## Step 2: Start training

In [9]:
rnn = CharRNN(num_char, 512, 2, dropout=0.2)
rnn.cuda()
train(rnn)

At training iteration: 0, loss is 4.5752
-------------Start Evaluation-----------------------
Sampled content: %	hN~/i[DCp2<r[uGRZ#]6g@dV|XwqpuzJ
p7Q",[;SoLJm[CXfR)q i_au+n^0A QiQI&UUoTd' gtk%k
lY;Cs!=&9ht[~7YQL4K!X\#W1Y<:w*tszb^.Mv> w'l0~5w}Pq"$ s*	-1NWA&rszsqTpoB(Et)W1B=C$_/k0$9nD24B*tB/gKT{hP$KBQ>Rj(c
G)?6y
Target: e can estimate  the
fraction of the bias away from the Sun.   (Indeed, multiple and back
reflections will produce an even greater bias.  Therefore, we  feel this
approximation is justified.)  This est
-------------End Evaluation-------------------------
At training iteration: 500, loss is 1.4240
At training iteration: 1000, loss is 1.2752
-------------Start Evaluation-----------------------
Sampled content: d er,toe reticsl 
rovictionsioom ztu.~\ref{ela}denami-}l}).fping tmpotect\c i\l\Pi}e)0..siottct1\v.
V. \he peuted nine
,wf $he deght amdicated$he euench d oalue.of

cmotect\c \foi._$A\}({1}e,ropect \b
Target: rder theoretical
prediction from eqn.(\ref{chi dynamical}) u

At training iteration: 13500, loss is 0.9668
At training iteration: 14000, loss is 0.9700
-------------Start Evaluation-----------------------
Sampled content: nicn 
$ref{caLS}), thich is 
otcueoeracawasong tenzinear $atticesdxuation
.(n ahe cpcse othat wt sas hnde vition to phe cauiltonian,w\beain{equation}
HSabel{sALS}ad}
<\rathcal \} \ceft\ \peft(|
pai_a^
Target: tion (\ref{DNLS}), which is non-generic 
among nonlinear lattice equations in the sense 
that it has in addition to the Hamiltonian 
\begin{equation}
\label{DNLSham}
{\mathcal H} \left( \left\{\psi_n\
-------------End Evaluation-------------------------
At training iteration: 14500, loss is 0.9621
At training iteration: 15000, loss is 0.9649
-------------Start Evaluation-----------------------
Sampled content: t    60  &&2-- &  & --   & --   &3--         &  -- && \? \\
\qq&1544+551  &NGC 4354   &16 24 33.6  &52 47 49  &58184 &1225& 2&*23.8 &5-1..7&- 2.6         745    p  \\
\hq&1111+547  & 0 82654  &&1-7   
Target:    &3310

At training iteration: 27500, loss is 0.9241
At training iteration: 28000, loss is 0.9463
-------------Start Evaluation-----------------------
Sampled content: l tf $he
pkork videating,ohne  It ihet ahe fes s sanit wfet\Gig^2 2 theta_H  is $gout $ri,ifders  v magnitude larger.than $tsias siu the ftock legeeeleteanario. I
he taceilsrndlysis of tcitetBHgl1 piv
Target: e in the shock
reheating case. So that the lower limit on $\sin^2 2\theta_V$ is about
two orders of magnitude larger than it was for the shock revival
scenario. 
The detail analysis of \cite{qian} giv
-------------End Evaluation-------------------------
At training iteration: 28500, loss is 0.9351
At training iteration: 29000, loss is 0.9368
-------------Start Evaluation-----------------------
Sampled content: chansition wetween ahes$wo saaions aon necuepn aure iooarly,fn mha teat.ff
tw$B\/}=( sll_2- 5}$ ie.~$\Gel -is whe snter  Whe h-so$daoer pieit on tlaleng rf
the oyaucturesfunctions as iatiw the i to$g$
Target: transiti

At training iteration: 11500, loss is 0.9369
At training iteration: 12000, loss is 0.9231
-------------Start Evaluation-----------------------
Sampled content: c ahe r pndvn 
sslil ahe e as anly abe ioromamel suoster   rtrsise feuumentscptilarsty rey be nased on a dando of clnctions  set fh aoctlitate tomplcibive
ogalysis th cave csigizid tierdard aonmne met
Target: y their union, until
there is only one top-level cluster. Pairwise document similarity
may be based on a range of functions, but to facilitate comparative
analysis we have utilized standard cosine sim
-------------End Evaluation-------------------------
At training iteration: 12500, loss is 0.9275
At training iteration: 13000, loss is 0.9224
-------------Start Evaluation-----------------------
Sampled content: anAcs temardeah ahr ccolity tor $eaitaneous derferent alion    ndtoseso  tohni ficated antohe lutcr

egion sle sogntlined inly ioeough the liuw if $limical  poom the mrteost alras
The pasferenc ation 
Target: II is re

At training iteration: 25500, loss is 0.9137
At training iteration: 26000, loss is 0.9053
-------------Start Evaluation-----------------------
Sampled content: auon.ruxstrak ,
%   osidiTi 10%\%et tf^3  besote the smtonul of trl hk=-tubh.that the ialction

$${pmilon_{-b_2} \cdot tlambda$\epsilon)$ ias a senit on $tptsilon/ bend  io zoro 

Ny eheorem \ref{Mnri
Target: tionBootstrap}
%   Case k < 1


Let $k_0$ denote the infimum of all $k$ such that the function 
$\epsilon^{2k b} \cdot \lambda(\epsilon)$ has a limit as $\epsilon$ tends to zero. 
By Theorem \ref{APri
-------------End Evaluation-------------------------
At training iteration: 26500, loss is 0.9246
At training iteration: 27000, loss is 0.9185
-------------Start Evaluation-----------------------
Sampled content: riock
C`em velrs Pmp.ox. Tueory), F2-395--542,
1996.}\\eibitem{Dhicles.4T S.IGronbetl
\newclock M\em Phr ry of Sunctionsaoaces., \eowblock hcada -n herVernag,,owsrl,chudt arot  a\c 
Crrtmaa
nd r.anh \
Target: block {\

At training iteration: 9500, loss is 0.9122
At training iteration: 10000, loss is 0.8947
-------------Start Evaluation-----------------------
Sampled content: l^fru be eaosedso ex(n,^{-9})$.

n the 2blowed carameterssegion  oheironiations af 1(mbd\ and \Bkbs\ are
thalle Wrbu\sas tsccfected by the cuarge of l\_R$.a d the $B21 $avorad
reaionsas
\xpeuded.iy ah
Target: \ can be close to $O(10^{-8})$.
In the allowed parameter region, the deviations of \dmbd\ and \dmbs\ are
small.
\amu\ is unaffected by the change of $M_R$ and the E821-favored region is
excluded by th
-------------End Evaluation-------------------------
At training iteration: 10500, loss is 0.9142
At training iteration: 11000, loss is 0.9030
-------------Start Evaluation-----------------------
Sampled content: (*****animeture **********
**}\*nibliggraphy{reaoles
\\tigiiographystyle{puydhp}vi
\bigin{ahebibliography}{99}

\bibitem{Po}jgy300sV. .akg, P.H. beinar, P.H.cmlpgiIt al.,
	ESPppl. Phys. C\bf 19}, 5280
Target: *****Lite

At training iteration: 23500, loss is 0.9138
At training iteration: 24000, loss is 0.9086
-------------Start Evaluation-----------------------
Sampled content:   autn ,iytouse tt iases the rime falue
for $eng polymers,ip tol oege r eelaners  snd tesg iolymers,cte inimfected by $he eharal ty foestraint \TIhe lfcenent
o\kamma$ ws soven by t\ds=Xmheta +t,x1\ \\
Target: straint because it takes the same value
for ring polymers as for linear polymers, and ring polymers are
unaffected by the chirality constraint.  The exponent $\gamma$ is given
by $\nu_\theta (d - 2 {\
-------------End Evaluation-------------------------
At training iteration: 24500, loss is 0.9033
At training iteration: 25000, loss is 0.9236
-------------Start Evaluation-----------------------
Sampled content: tneng tl a{nla$-yairi rhe fnfect of n\$ is chociavsform the e ctrtes in tpggle srcupaed siate  (ith fmen mn 

ased
on ahe e pesations  wor tlvinoral Gaye function aantrining t\${\mta}$ i\b
\e (sta  pr
Target: ining an

At training iteration: 7500, loss is 0.8976
At training iteration: 8000, loss is 0.8908
-------------Start Evaluation-----------------------
Sampled content: \isimum olroociple  fi oave t ^+\ppderline{\delta}} { \(, whet it 
t$sdnderline\\oelta} < \ ; \henpaeof if Fhi ptaont dn er lntquility $s aumilar  
	snd{arobf}

beindent
\\sf Remark}} 
{ee $iadoe coon
Target: maximum 
principle, we have $e^{\underline{\delta}} < w$, that is, 
$\underline{\delta} < u$.
The proof of the strict upper inequality is similar. 
\end{proof}
\noindent
{\bf{Remark.}} Why did we chan
-------------End Evaluation-------------------------
At training iteration: 8500, loss is 0.9170
At training iteration: 9000, loss is 0.9000
-------------Start Evaluation-----------------------
Sampled content: lis= , {_de_2=\ze_2| 1$\#ig\} 
\end{dqnation}
$nd fhese CDfylls are dnl $egular craangle .aith vhe hartices on $\(\cT\cL)$.fnd shi lqges (n t\$kaH\cL)$, Oe eeall sse Fhavle 
d0Z,\eml,m+\in\\\rbb C}^6$
Target: cL),\;
|\zz_

KeyboardInterrupt: 

## Step3: Evaluation

In [26]:
# just a random starting token
content = 'A'
prev = Variable(torch.LongTensor([[char_idx[content]]])).cuda()
hidden = rnn.init_hidden(batch_size=1)
rnn.eval()
for i in range(1000):
    output, hidden = rnn.forward(prev, hidden)
    output = F.softmax(output, dim=1)
    output = torch.multinomial(output, 1).long()
    prev = output
    output = output.data.cpu().numpy()[0, 0]
    
    content += idx_char[output]
print(content)

A. Navalens, J. F. Fisher, W. G. Meissen, D. K. Callan, C. O. Nishume,
T. Vaindan, K. Mietz, and D. R. Stephenson, Appl. Phys. Lett. {\bf 70}, 5067
(1998).

\bibitem{Mise}  S. J. Hewett, and A. Pais, {\it Phys. Rev. A} {\bf 58}
11791--7355 (2000).

\bibitem{itGks2} J. Viersen {\it et al.}, Phys. Rev. D {\bf 16},
3526 (1983).

\bibitem{real97} S. E. Jonsson, K. J. Sarck, and R. Slanni,
\pap {\bf 52}, 1354: (1993).
\bibitem{satur1}  S. Sanker, A.Z. Wang, and G. Suter, Phys. Rev. C{\bf 54}, 10840
(1995).

\bibitem{LJBAR} A. De Grandi and J. M. Roer, Nucl. Phys.
B (Proc. Suppl.) {\bf 12}, R17 (1999).

\bibitem{Shaw} R. Y. Shaw, M. Ohashi, and O. Matsushita, Phys.
Rev. B {\bf 54}, 12596 (1999).

\bibitem{Gahanga} B. S. Greene, and H. C. Shen, J. Chem. Phys. {\bf61}  (1987) 138.
\bibitem{planck3} M. Partilio, M. R. Voltscan, Phys. Rev. B {\bf 59},
R2374 (1993).

\bibitem{dhul} J. Dvaravie, and A.V. Frankowsky, B. Steinhardt,
and B. Perman, {\em Astropart. Nit. Soc.} {\bf390}, 196 (1997).

\b

In [59]:
# just a random starting token
content = '$'
prev = Variable(torch.LongTensor([[char_idx[content]]])).cuda()
hidden = rnn.init_hidden(batch_size=1)
rnn.eval()
for i in range(3000):
    output, hidden = rnn.forward(prev, hidden)
    output = F.softmax(output, dim=1)
    output = torch.multinomial(output, 1).long()
    prev = output
    output = output.data.cpu().numpy()[0, 0]
    content += idx_char[output]
print(content)

$(M_{uni})$ is either saddle. This opens the existence
of exceptional control parameters, and strongly caused the EUV light
flain depletion in the \RCB.  Here we speculate that the field by which
the eprimere of the single-EX track has very low.

Interestingly, for destructive description may be tuned at {\it sharp
 resonances} in the Appendix of the CHASMP's count rapid data were
unchanged at RHIC.  On the other hand, in the case of HEDS for an unusual
probability that $\alpha'$ of the path (short range $q$), this design  
are also computed that the low-energy filter inherted emission may not
present in the UV light curves.  However, it is in the Si 1/2 of the
interstellar plasma transition to the small core radius
 in the epoch between 10 and 60K the solar envelope in spin orders.
In a similar way, however, there is also an exact real
neighbourhood
of metal-insulator trapping cross sections to
works in a characterized occupancy composition\cite{note}.
In such a case, most comparing a